In [3]:
pip install geocoder

     |████████████████████████████████| 102kB 4.5MB/s ta 0:00:011
     |████████████████████████████████| 92kB 19.4MB/s eta 0:00:01
     |████████████████████████████████| 829kB 26.0MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Restart kernel

In [1]:
# Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.
# We will use the Geocoder Python package: https://geocoder.readthedocs.io/index.html.
import pandas as pd 
import wikipedia as wp
import geocoder
import requests
import io

In [2]:
# initialize your variable to None
lat_lng_coords = None

In [3]:
# Download url from Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")

In [4]:
# determine the index of your table
df = pd.read_html(html, header = 0)[0]

In [6]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df = df[df.Borough != 'Not assigned']

In [2]:
# Download url from Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")

In [3]:
# Determine table index
df = pd.read_html(html, header = 0)[0]

In [7]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
for index, row in df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']

In [8]:
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [9]:
# Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data
url="http://cocl.us/Geospatial_data"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

In [13]:
df

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
...,...,...,...
160,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing CentrE
169,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...


In [15]:
# Rename the first column to allow merging dataframes on Postal code
c.columns = ['Postal code', 'Latitude', 'Longitude']
df = pd.merge(c, df, on='Postal code')

# Reorder column names and show the dataframe
df = df[['Postal code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
df

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...,43.688905,-79.554724
101,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...,43.739416,-79.588437


This dataframe now has 103 distinct Postal codes from the Toronto Canada area and now has the lat and long added to total 5 columns.